In [2]:
from datasets import load_dataset, load_from_disk

## 加载结果数据集

In [3]:
## repairllama
file_path='/Users/alex.wu/PycharmProjects/repairllama/results/defects4j/repairllama/lora/RepairLLaMA_defects4j_f2f_bugs_results_ir4_or2.jsonl'
repairllama_dataset=load_dataset('json', data_files=file_path)['train']

Found cached dataset json (/Users/alex.wu/.cache/huggingface/datasets/json/default-086715474a996e88/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
## ours

ours_dataset=load_from_disk('/Users/alex.wu/PycharmProjects/apr_datasets_processing/defects4j_validation/dataset_validated/defects4j_context_gen_10_validation')

def name_map(x):
    x['bug_id']=x['projectName'].capitalize().replace('Jacksondatabind', 'JacksonDatabind').replace('Jacksoncore', 'JacksonCore').replace(
            'Jacksonxml', 'JacksonXml').replace('Jxpath', 'JxPath').replace('_', '-')
    x['gen']=[i.replace(x['input'], '').replace('<s>','').replace('</s>','').replace('<unk>','') for i in x['gen']]
    for i in x['test_res']:
        del i['test_message']
    return x

ours_dataset=ours_dataset.map(name_map)

Parameter 'function'=<function name_map at 0x7ff4c81d8310> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/223 [00:00<?, ? examples/s]

In [6]:
import re

def remove_java_comments_correctly(code):
    """
    正确地去除Java代码中的单行和多行注释，不移除注释前的换行符。

    参数:
    - code: 字符串，包含Java代码。

    返回:
    - 去除注释的代码字符串，保留其他内容不变。
    """
    # 移除多行注释，这里假设多行注释不跨越多个非空白行
    code_no_multiline_comments = re.sub(r'/\*[\s\S]*?\*/', '', code)
    # 移除单行注释及其后的换行符，如果存在
    cleaned_code = re.sub(r'(?m)^ *//.*\n?', '', code_no_multiline_comments)

    return cleaned_code.strip()

def remove_spaces_newlines_and_get_indices(java_code):
    cleaned_code = ""
    indices = []
    for index, char in enumerate(java_code):
        if char not in [' ', '\n', '\r', '\t']:
            cleaned_code += char
            indices.append(index)
    return indices, cleaned_code

def find_substring_indices(main_string, substring):
    start_index = main_string.find(substring)
    
    # 如果找不到子字符串，则返回-1
    if start_index == -1:
        return -1, -1

    end_index = start_index + len(substring) - 1
    return start_index, end_index

def replace_sub(java_code, java_patch):
    rm_code_ind_lst, rm_java_code=remove_spaces_newlines_and_get_indices(java_code)
    _, rm_java_patch=remove_spaces_newlines_and_get_indices(java_patch)
    start_ind, end_ind=find_substring_indices(rm_java_code, rm_java_patch)
    code_start_ind, code_end_ind=rm_code_ind_lst[start_ind], rm_code_ind_lst[end_ind]
    patch=java_code[:code_start_ind]+java_code[code_end_ind+1:]
    return patch

In [7]:
def unify(x):
    bug_id=x['bug_id']
    repairllama_sample=repairllama_dataset.filter(lambda x:x['bug_id']==bug_id)[0]
    res=[]
    for i,patch in enumerate(repairllama_sample['patches']):
        patch=remove_java_comments_correctly(patch)
        before_code=x['buggyInfo']['methodPreContext'] 
        after_code=x['buggyInfo']['methodPostContext'] 
        patch=replace_sub(patch, before_code)
        patch=replace_sub(patch, after_code)
        d={'correctness':repairllama_sample['test_results'][i].lower().replace('test fail','wrong').replace('compile fail','uncompilable'),
          'patch':patch.strip()}
        res.append(d)
    x['repairllama_res']=res
    return x

In [ ]:
dataset=ours_dataset.map(unify)

## repairllama 统计情况

In [ ]:
match={}
ast_match={}
plausible={}
wrong={}
uncompilable={}
for i in dataset:
    bug_id=i['bug_id']
    res=i['repairllama_res']
    d={}
    for j in res:
        key=j['correctness']
        if key in d:
            d[key].append(j['patch'])
        else:
            d[key]=[j['patch']]
    if 'line match' in d:
        match[bug_id]=d['line match']
        print(f'\n\n==========={bug_id}================')
        print(i['input'].split('// method to be repaired')[-1])
        print('\nfix: ', match[bug_id][0])
    elif 'ast match' in d:
        ast_match[bug_id]=d['ast match']
    elif 'plausible' in d:
        plausible[bug_id]=d['plausible']
    elif 'wrong' in d:
        wrong[bug_id]=d['wrong']
    elif 'uncompilable' in d:
        uncompilable[bug_id]=d['uncompilable']
        
print(len(match))
print(len(plausible))
print(len(wrong))
print(len(uncompilable))

In [16]:
def get_avg_line_number(match):
    line_lst=[]
    for i in match.values():
        line_lst+=i
    len_lst=[len(i.split('\n')) for i in line_lst]
    return sum(len_lst)/len(len_lst)

In [17]:
get_avg_line_number(match)

1.7469879518072289

In [18]:
repair_lst=[]
v1_lst=[]
for i in dataset:
    for j in i['repairllama_res']:
        repair_lst.append(j['patch'])
    for j in i['test_res']:
        v1_lst.append(j['patch'])  
len_repair_lst=[len(i.split('\n')) for i in repair_lst]
len_v1_lst=[len(i.split('\n')) for i in v1_lst]
print(sum(len_repair_lst)/len(len_repair_lst))
print(sum(len_v1_lst)/len(len_v1_lst))

2.4269058295964125
2.939521800281294


## ours 统计情况

In [19]:
ours_plausible={}
ours_wrong={}
ours_uncompilable={}
for i in dataset:
    bug_id=i['bug_id']
    res=i['test_res']
    d={}
    for j in res:
        key=j['correctness']
        if key in d:
            d[key].append(j['patch'])
        else:
            d[key]=[j['patch']]

    if 'plausible' in d:
        ours_plausible[bug_id]=d['plausible']
        # print(f'-----{bug_id}------')
        # print('fix: ', i['buggyInfo']['originalCode'])
        # print(d['plausible'][0])
    elif 'wrong' in d:
        ours_wrong[bug_id]=d['wrong']
    elif 'uncompilable' in d:
        ours_uncompilable[bug_id]=d['uncompilable']
        
print(len(ours_plausible))
print(len(ours_wrong))
print(len(ours_uncompilable))

25
127
68


In [20]:
get_avg_line_number(ours_plausible)

3.3043478260869565

#### unique pluasible patch

In [29]:
from collections import ChainMap
set(ours_plausible.keys()).difference(set(ChainMap(match, plausible).keys()))

{'Closure-126',
 'Compress-1',
 'Csv-4',
 'JacksonDatabind-107',
 'JxPath-21',
 'JxPath-8',
 'Math-89',
 'Math-9',
 'Mockito-12'}

## 查看利用class上下文情况

In [133]:
def get_member_name(string):
    return string.split(' ')[-1]

def get_method_name(string):
    return string.split('(')[0].split(' ')[0]

def is_class_context_used(x, name):
    inputs=x['input'].split('// method to be repaired\n')[1]
    member_lst=[get_member_name(i) for i in x['classInformation']['memberVariables']]
    method_lst=[get_method_name(i) for i in x['classInformation']['memberMethods']]
    
    unused_context_lst=[i for i in member_lst+method_lst if i not in inputs]
    
    patches=[i['patch'] for i in x[name]]
    for i in unused_context_lst:
        if i in '\n'.join(patches):
            return True
    return False        

In [134]:
## ours利用上下文
class_context_used_dataset=dataset.filter(lambda x:is_class_context_used(x, 'test_res'))
print(len(class_context_used_dataset))

Filter:   0%|          | 0/223 [00:00<?, ? examples/s]

28


In [138]:
class_context_used_dataset

Dataset({
    features: ['methodInformation', 'involvedTypesInformation', 'filePath', 'classInformation', 'buggyInfo', 'projectName', 'input', 'output', 'gen', 'test_res', 'bug_id', 'repairllama_res'],
    num_rows: 28
})

In [135]:
## repairllama利用上下文 
print(len(dataset.filter(lambda x:is_class_context_used(x, 'repairllama_res'))))

Filter:   0%|          | 0/223 [00:00<?, ? examples/s]

11


In [ ]:
for sample in class_context_used_dataset:
    bug_id=sample['bug_id']
    res=sample['test_res']
    fix=sample['buggyInfo']['originalCode']
    print(f'==========={bug_id}================')
    print(sample['input'])
    print('fix: ', fix)
    print('---ours repair------')
    for i in res:
        print(i)
    print('---repairllama------')
    for i in sample['repairllama_res']:
        print(i)

## 查看整体patch情况

In [11]:
for sample in dataset:
    bug_id=sample['bug_id']
    res=sample['test_res']
    fix=sample['buggyInfo']['originalCode']
    print(f'==========={bug_id}================')
    print(sample['input'])
    print('fix: ', fix)
    print('---ours repair------')
    for i in res:
        print(i)
    print('---repairllama------')
    for i in sample['repairllama_res']:
        print(i)

===========Cli-4================
<PRE> // class information of method
public class Parser implements CommandLineParser{
    private CommandLine cmd
    private Options options
    private List requiredOptions
    protected abstract String[] flatten(Options opts, String[] arguments, boolean stopAtNonOption)
    public CommandLine parse(Options options, String[] arguments)
    public CommandLine parse(Options options, String[] arguments, Properties properties)
    public CommandLine parse(Options options, String[] arguments, boolean stopAtNonOption)
    public CommandLine parse(Options options, String[] arguments, Properties properties, boolean stopAtNonOption)
    private void processProperties(Properties properties)
    private void checkRequiredOptions()
    public void processArgs(Option opt, ListIterator iter)
    private void processOption(String arg, ListIterator iter)
}

// method to be repaired
    private void checkRequiredOptions()
        throws MissingOptionException
    {
 

In [10]:
dataset.filter(lambda x:x['bug_id']=='Cli-4')[0]['gen']

Filter:   0%|          | 0/223 [00:00<?, ? examples/s]

[' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();',
 ' StringBuffer buff = new StringBuffer();']